In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as df
%matplotlib inline

In [24]:
train_original = pd.read_csv('train.csv')
validation_original = pd.read_csv('validation.csv')
test_original = pd.read_csv('test.csv')

In [25]:
# split os and browser
def split_useragent_os(data):
    return data.split('_')[0]
def split_useragent_b(data):
    return data.split('_')[1]
def split_useragent(dataframe):
    dataframe['os'] = dataframe['useragent'].apply(split_useragent_os)
    dataframe['browser'] = dataframe['useragent'].apply(split_useragent_b)
    dataframe.drop(['useragent'], axis=1, inplace=True)

In [26]:
# original data now has os and browser columns
split_useragent(train_original)
split_useragent(validation_original)
split_useragent(test_original)

In [33]:
#drop meaningless columns and the columns not in testdata
train_to_drop = ['city','slotid','slotprice','IP','domain','usertag','creative','bidid','userid','url','urlid','bidprice','payprice','keypage']
test_to_drop = ['city','slotid','slotprice','IP','domain','usertag','creative','bidid','userid','url','urlid','keypage']
train_drop = train_original.drop(train_to_drop, axis=1)
validation_drop = validation_original.drop(train_to_drop, axis=1)
test_drop = test_original.drop(test_to_drop, axis=1)

In [34]:
#make the catagorical data into one hot format : eg monday ->[ 1, 0, 0 ,0 ,0 ,0 ,0 ]
def onehot(onehot_df, column_name):
    dummies = pd.get_dummies(onehot_df[column_name],prefix=column_name)
    onehot_df = onehot_df.join(dummies)
    onehot_df = onehot_df.drop([column_name], axis=1)
    return onehot_df
column_to_onehot = ['weekday','hour','region','adexchange','slotformat','os','browser','slotwidth','slotheight','slotvisibility']
for col_name in column_to_onehot:
    train_drop = onehot(train_drop, col_name)
    validation_drop = onehot(validation_drop, col_name)
    test_drop = onehot(test_drop , col_name)

In [ ]:
# def getAllUserTag():
#     taglist = []
#     for tags in df_train_original["usertag"].values:
#         #print(type(tags))
#         if type(tags)is str:
#             t = tags.split(',')
           
#             taglist.extend(t)
#     tagset = set(taglist)
#     return tagset
# tagset = getAllUserTag()


# def addUserTagOneHot(sourceData,tagset):
#     sourceData = sourceData.copy()
#     usertagColumn = sourceData["usertag"].values
#     for tag in tagset:
#         sourceData[tag]=0

#     return sourceData
# addZeroTag = addUserTagOneHot(df_train_original,tagset)

# usertagColumn = addZeroTag["usertag"].values
# for i in range(len(usertagColumn)):
#     tags = usertagColumn[i]
#     if type(tags)is str:
#         t = tags.split(',')
#         for tag in t:
#             addZeroTag.iloc[i][t] =1

In [29]:
train_usertag = pd.read_csv('train_usertag.csv')
validation_usertag = pd.read_csv('validation_usertag.csv')
test_usertag = pd.read_csv('test_usertag.csv')

In [48]:
#train_drop = pd.concat([train_drop, train_usertag], axis=1, join_axes=[train_drop.index])
test_drop = pd.concat([test_drop, test_usertag], axis=1, join_axes=[test_drop.index])
val_drop = pd.concat([validation_drop, validation_usertag], axis=1, join_axes=[validation_drop.index])

In [37]:
train_drop[:5]

,click,advertiser,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,hour_0,...,13403,10133,10074,13678,10145,10076,16617,13042,11632,10110
0,0,3427,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2821,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3427,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1458,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2259,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
def evaluate(data):
    buyyed = data[data['bidprice']>= data['payprice']]
    ctr,clicks,cost = buyyed['click'].mean(),buyyed['click'].sum(),buyyed['payprice'].sum()
    CPC = cost/clicks/1000
    CPM = cost /buyyed['click'].count() *1000
    return ctr,clicks,cost,CPC,CPM

In [41]:
#get two key parameter in the linear bidding function 
avgCTR1= train_original['click'].mean()
avgCTR2= validation_original['click'].mean()
# bidbase = df_train_original['payprice'].mean()

In [83]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier  
RF_clf = RandomForestClassifier(n_estimators=200)
RF_clf.fit(train_drop.drop(['click'],axis = 1), train_original['click'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [84]:
RF_clf_r = RF_clf.predict_proba(train_drop.drop(['click'],axis = 1).values[:])
#training set testing 
rf_clf_trainP = pd.DataFrame(RF_clf_r)
rf_clf_trainP['bidprice'] = rf_clf_trainP[1]
rf_clf_trainP['unitprice'] = rf_clf_trainP['bidprice'] /avgCTR 

rf_clf_trainP['bidprice'] =rf_clf_trainP['bidprice'] /avgCTR * 100
rf_clf_trainP['payprice'] = train_original['payprice']
rf_clf_trainP['click'] = train_original['click']

In [85]:
evaluate(rf_clf_trainP)

(0.026362994765629595, 1793, 5830544, 3.2518371444506413, 85728.165617832143)

In [87]:
val_RF_clf_r = RF_clf.predict_proba(val_drop.drop(['click'],axis = 1).values[:])

In [88]:
#tvalidation set testing

val_rf_clf_trainP = pd.DataFrame(val_RF_clf_r)
val_rf_clf_trainP['bidprice'] = val_rf_clf_trainP[1]
val_rf_clf_trainP['unitprice'] = val_rf_clf_trainP['bidprice'] /avgCTR2

val_rf_clf_trainP['bidprice'] =val_rf_clf_trainP['bidprice'] /avgCTR2 * 100
val_rf_clf_trainP['payprice'] = validation_original['payprice']
val_rf_clf_trainP['click'] = validation_original['click']

In [89]:
evaluate(val_rf_clf_trainP)

(0.004240095402146548, 96, 1909285, 19.88838541666667, 84328.651561326798)

In [90]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = validation_original['click'].values
y_pre = val_rf_clf_trainP['bidprice']
auc_val_xg1 = roc_auc_score(y_true, y_pre)
auc_val_xg1

0.71363054461134023

In [86]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = train_original['click'].values
y_pre = rf_clf_trainP['bidprice']
auc_val_xg1 = roc_auc_score(y_true, y_pre)
auc_val_xg1

1.0

In [ ]:
###LogisticRegression

In [91]:
from sklearn.linear_model import LogisticRegression as lr
from sklearn import metrics
lr_instance = lr()
lr_instance.fit(train_drop.drop(['click'],axis = 1), train_original['click'])
testProbability = lr_instance.predict_proba(train_drop.drop(['click'],axis = 1).values[:])

In [96]:
testProbability = pd.DataFrame(testProbability)
testProbability['bidprice'] = testProbability[1]
testProbability['bidprice'] = 97 * testProbability['bidprice'] /avgCTR 

testProbability['payprice'] = train_original['payprice']
testProbability['click'] = train_original['click']

In [97]:
evaluate(testProbability )

(0.0006417989386547181, 432, 39409149, 91.22488194444443, 58548.032410846405)

In [98]:
y_true = train_original['click'].values
y_pre = testProbability['bidprice']
auc_val_xg1 = roc_auc_score(y_true, y_pre)
auc_val_xg1

0.50333030294798631

In [100]:
val_testProbability = lr_instance.predict_proba(val_drop.drop(['click'],axis = 1).values[:])
val_lr_clf = pd.DataFrame(val_testProbability)
val_lr_clf['bidprice'] = val_lr_clf[1]
val_lr_clf['unitprice'] = val_lr_clf['bidprice'] /avgCTR2 

val_lr_clf['bidprice'] =val_lr_clf['bidprice'] /avgCTR2 * 100
val_lr_clf['payprice'] = validation_original['payprice']
val_lr_clf['click'] = validation_original['click']

In [101]:
evaluate(val_lr_clf )

(0.0005715726653778263, 68, 7425548, 109.19923529411766, 62415.29797427923)

In [102]:
y_true = validation_original['click'].values
y_pre = val_lr_clf['bidprice']
auc_val_xg1 = roc_auc_score(y_true, y_pre)
auc_val_xg1

0.53747480564869832

In [103]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(train_drop.drop(['click'],axis = 1), train_original['click'])

gb_val_proba = gb_clf.predict_proba(val_drop.drop(['click'],axis = 1).values[:])

val_gb_clf_trainP = pd.DataFrame(gb_val_proba)
val_gb_clf_trainP['bidprice'] = val_gb_clf_trainP[1]
val_gb_clf_trainP['unitprice'] = val_gb_clf_trainP['bidprice'] /avgCTR2 

val_gb_clf_trainP['bidprice'] = 90 * val_gb_clf_trainP['bidprice'] /avgCTR2
val_gb_clf_trainP['payprice'] = validation_original['payprice']
val_gb_clf_trainP['click'] = validation_original['click']

In [105]:
evaluate(val_gb_clf_trainP )

(0.0008915746198424884, 108, 4478452, 41.46714814814815, 36971.05684613734)

In [106]:
y_true = validation_original['click'].values
y_pre = val_gb_clf_trainP['bidprice']
auc_val_xg1 = roc_auc_score(y_true, y_pre)
auc_val_xg1MJ.                                                                        

0.6540158253239019

In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

from xgboost.sklearn import XGBClassifier 
xgb_clf = XGBClassifier(
    ooster='gbtree',
    learning_rate = 0.5,
    n_estimators=50,
    max_depth=10,
    min_child_weight=9,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.85,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1, 
    seed=27,
    n_jobs=4)





In [112]:
validation_clf = pd.DataFrame(xgb_clf)
validation_clf['bidprice'] = validation_clf[1]
validation_clf['unitprice'] = validation_clf['bidprice'] /avgCTR2 

validation_clf['bidprice'] = 97 * validation_clf['bidprice'] /avgCTR2
validation_clf['payprice'] = validation_original['payprice']
validation_clf['click'] = validation_original['click']

NameError: name 'val_clf' is not defined

In [ ]:
y_true = validation_original['click'].values
y_pre = validation_clf['bidprice']
auc_val_xg1 = roc_auc_score(y_true, y_pre)
auc_val_xg1MJ.  